In [ ]:
from typing import Any, Mapping, Sequence, cast
import math
from collections import defaultdict

import numpy as np
from pymatgen.core import Structure

from smol.cofe import ClusterExpansion

# ---- phaseedge imports (modern-only) ----
from phaseedge.jobs.store_ce_model import lookup_ce_by_key
from phaseedge.jobs.train_ce import (
    featurize_structures,
    predict_from_features,
    compute_stats,
    _n_replace_sites_from_prototype,
)
from phaseedge.schemas.calc_spec import CalcSpec
from phaseedge.schemas.mixture import Mixture, composition_map_sig, sublattices_from_mixtures
from phaseedge.science.prototype_spec import PrototypeSpec
from phaseedge.storage.cetrainref_dataset import Dataset
from phaseedge.storage.store import lookup_total_energy_eV
from phaseedge.utils.keys import occ_key_for_structure


# ----------------- helpers: structures, sources, sublattices -----------------

def fmt_mev(x: float | str) -> str:
    if isinstance(x, str):
        return x
    return f"{1e3 * float(x):.3f}"

def _gather_composition_mixtures(sources: Sequence[Mapping[str, Any]]) -> list[Mixture]:
    mixes: list[Mixture] = []
    for s in sources:
        if str(s.get("type", "")).lower() != "composition":
            continue
        for m in s.get("mixtures", []):
            mixes.append(
                Mixture.from_dict(
                    {
                        "composition_map": dict(m["composition_map"]),
                        "K": int(m["K"]),
                        "seed": int(m["seed"]),
                    }
                )
            )
    if not mixes:
        raise RuntimeError("Expected 'composition' mixtures in sources (modern schema).")
    return mixes


def _sublattices_for_doc(doc: Mapping[str, Any]) -> dict[str, tuple[str, ...]]:
    """
    Determine replaceable sublattices for per-site scaling:
      - composition CE: from this doc's mixtures
      - dopt sampling CE : from the base (composition) CE's mixtures
    """
    sources = cast(Sequence[Mapping[str, Any]], doc["sources"])
    if not sources:
        raise RuntimeError("CE document missing 'sources'.")
    s0 = sources[0]
    s_type = str(s0.get("type", "")).lower()

    if s_type == "composition":
        return sublattices_from_mixtures(_gather_composition_mixtures(sources))

    if s_type == "dopt_sampling_intent":
        base_ce_key = cast(str, s0["base_ce_key"])
        base_doc = lookup_ce_by_key(base_ce_key)
        if not base_doc:
            raise RuntimeError(f"Base CE not found for dopt_sampling_intent source: {base_ce_key}")
        return sublattices_from_mixtures(_gather_composition_mixtures(cast(Sequence[Mapping[str, Any]], base_doc["sources"])))

    raise RuntimeError(f"Unsupported source type in CE doc: {s_type!r}")

# ----------------- pretty printers ------------------------------------------

def _print_design_metrics(dm: Mapping[str, Any] | None) -> None:
    if not dm:
        print("Design matrix diagnostics: <not available>")
        return

    def _get(name: str, default: float | int | str = float("nan")) -> Any:
        return dm.get(name, default)

    try:
        n_samples = int(_get("n_samples", -1))
        n_features = int(_get("n_features", -1))
        rank = int(_get("rank", -1))

        sigma_max = float(_get("sigma_max", float("nan")))
        sigma_min = float(_get("sigma_min", float("nan")))
        kappa = float(_get("condition_number", float("inf")))
        logdet = float(_get("logdet_xtx", float("nan")))

        lev_mean = float(_get("leverage_mean", float("nan")))
        lev_max = float(_get("leverage_max", float("nan")))
        lev_p95 = float(_get("leverage_p95", float("nan")))

        weighting_applied = bool(_get("weighting_applied", False))
        standardization = str(_get("standardization", "none"))
        zero_var = int(_get("zero_variance_feature_count", 0))
    except Exception:
        print("Design matrix diagnostics: <malformed in CE document>")
        return

    print("Design matrix diagnostics (stored)")
    print("  Samples / Features     : %d / %d" % (n_samples, n_features))
    print("  Rank                   : %d" % rank)
    print("  Sigma max / min        : %.6g / %.6g" % (sigma_max, sigma_min))
    if math.isfinite(kappa):
        print("  Condition number (κ)   : %.6g" % kappa)
    else:
        print("  Condition number (κ)   : inf (rank-deficient or σ_min≈0)")
    print("  log det(XᵀX)           : %.6g" % logdet)
    print("  Leverage mean / p95/max: %.6g / %.6g / %.6g" % (lev_mean, lev_p95, lev_max))
    print("  Weighting applied      : %s" % weighting_applied)
    print("  Standardization        : %s" % standardization)
    print("  Zero-variance columns  : %d" % zero_var)


def _print_mixture_energy_ranges(
    mixture_rows: list[tuple[str, int, float, float, float, float]],
) -> None:
    """
    Each row: (mixture_key, n, E_min_prim, E_max_prim, dE_prim, dE_meV_per_site)
    """
    print("-" * 72)
    print("Mixture energy span (training set)".center(72))
    print("-" * 72)
    hdr = (
        f"{'Mixture (Es/Fm counts)':<42}  {'n':>3}  "
        f"{'E_min (eV/prim)':>14}  {'E_max (eV/prim)':>16}  {'ΔE (eV/prim)':>13}  {'ΔE (meV/site)':>15}"
    )
    print(hdr)
    print("-" * 72)
    for key, n, e_min, e_max, de_prim, de_mev_site in mixture_rows:
        print(
            f"{key:<42}  {n:>3d}  "
            f"{e_min:>14.6f}  {e_max:>16.6f}  {de_prim:>13.6f}  {de_mev_site:>15.3f}"
        )
    print("-" * 72)


def _print_summary(
    *,
    ce_key: str,
    n: int,
    stored_in: Mapping[str, Any],
    recomputed_in: Mapping[str, Any],
    stored_cv: Mapping[str, Any] | None,
    design_metrics: Mapping[str, Any] | None,
    mixture_rows: list[tuple[str, int, float, float, float, float]],
    atol: float,
    rtol: float,
    ok: bool,
) -> None:
    s_mae, s_rmse, s_max = (
        float(stored_in["mae_per_site"]),
        float(stored_in["rmse_per_site"]),
        float(stored_in["max_abs_per_site"]),
    )
    r_mae, r_rmse, r_max = (
        float(recomputed_in["mae_per_site"]),
        float(recomputed_in["rmse_per_site"]),
        float(recomputed_in["max_abs_per_site"]),
    )

    print("═" * 72)
    print(f" CE Validation (per-site metrics in meV) ".center(72, "═"))
    print("═" * 72)
    print(f"CE key        : {ce_key}")
    print(f"samples (n)   : {n}")
    print(f"tolerances    : atol={atol:g}, rtol={rtol:g}")
    print("-" * 72)
    print("In-sample")
    print("Metric         Stored    Recomputed   Δ (recomp − stored)")
    print("-------------  --------  ----------   -------------------")
    print(f"MAE            {fmt_mev(s_mae):>8}   {fmt_mev(r_mae):>8}        {fmt_mev(r_mae - s_mae):>8}")
    print(f"RMSE           {fmt_mev(s_rmse):>8}   {fmt_mev(r_rmse):>8}        {fmt_mev(r_rmse - s_rmse):>8}")
    print(f"Max|err|       {fmt_mev(s_max):>8}   {fmt_mev(r_max):>8}        {fmt_mev(r_max - s_max):>8}")
    print("-" * 72)

    if stored_cv and all(k in stored_cv for k in ("n", "mae_per_site", "rmse_per_site", "max_abs_per_site")):
        cv_n = int(stored_cv["n"]); cv_mae = float(stored_cv["mae_per_site"])
        cv_rmse = float(stored_cv["rmse_per_site"]); cv_max = float(stored_cv["max_abs_per_site"])
        print("Stored 5-fold CV")
        print("Metric         Value")
        print("-------------  --------")
        print(f"n              {cv_n}")
        print(f"MAE            {fmt_mev(cv_mae):>8}")
        print(f"RMSE           {fmt_mev(cv_rmse):>8}")
        print(f"Max|err|       {fmt_mev(cv_max):>8}")
        print("-" * 72)
    else:
        print("Stored 5-fold CV: <not available>")
        print("-" * 72)

    _print_design_metrics(design_metrics)

    if mixture_rows:
        _print_mixture_energy_ranges(mixture_rows)

    print("Result         : " + ("✅  PASS (in-sample matches stored)" if ok else "❌  FAIL (in-sample mismatch)"))
    print("═" * 72)


# ----------------- main: in-sample validator + mixture spans -----------------

def verify_ce_in_sample(
    ce_key: str,
    *,
    atol: float = 1e-6,
    rtol: float = 1e-8,
    verbose: bool = True,
) -> bool:
    """
    Validate CE *in-sample* stats using the serialized Structures stored in train_refs.
    Also print per-mixture energy ranges (E_min, E_max, ΔE) by inferring Es/Fm
    sublattice counts from O-coordination (4 vs 6).
    """
    # ---- Load CE document (modern schema assumed) ----
    doc = lookup_ce_by_key(ce_key)
    if not doc:
        raise RuntimeError(f"No CE found for ce_key={ce_key}")

    sx, sy, sz = (int(x) for x in doc["supercell_diag"])
    supercell_diag = (sx, sy, sz)

    stats = cast(Mapping[str, Any], doc["stats"])
    in_sample_stored = cast(Mapping[str, Any], stats["in_sample"])
    stored_cv = cast(Mapping[str, Any] | None, stats.get("five_fold_cv"))

    # Design metrics (stored at training time)
    design_metrics = cast(Mapping[str, Any] | None, doc.get("design_metrics"))

    # sublattices for per-site scaling
    sublattices = _sublattices_for_doc(doc)
    # Cation symbols we care about (e.g., {"Al","Mg"})
    cation_symbols: set[str] = set()
    for _, allowed in sublattices.items():
        for el in allowed:
            cation_symbols.add(str(el))
    if not cation_symbols:
        raise RuntimeError("Empty cation symbol set inferred from sublattices.")

    # ---- Gather training Structures and energies in the recorded order ----
    dataset = Dataset.from_key(doc["dataset_key"])

    # Targets are per primitive/conventional cell
    n_prims = int(np.prod(np.asarray(supercell_diag, dtype=int)))
    calc_spec_top = CalcSpec.from_dict(doc["calc_spec"])

    structures: list[Structure] = []
    y_true_per_prim: list[float] = []
    by_composition_map: dict[str, list[float]] = defaultdict(list)
    for i, r in enumerate(dataset.train_refs):
        composition_map = r.composition_map
        occ_key = r.occ_key
        calc_spec = r.calc_spec

        if calc_spec_top != calc_spec:
            raise RuntimeError(
                f"train_refs[{i}] calc_spec identity differs from CE doc: "
                f"{calc_spec} != {calc_spec_top}"
            )

        # Strong sanity: recompute occ_key from structure and compare
        ok2 = occ_key_for_structure(r.structure)
        if ok2 != occ_key:
            raise RuntimeError(
                f"train_refs[{i}] occ_key mismatch: expected {occ_key}, rebuilt {ok2}. "
                "This indicates a change in structure hashing."
            )

        structures.append(r.structure)
        energy_result = lookup_total_energy_eV(
            occ_key=occ_key,
            calc_spec=calc_spec,
        )
        if energy_result is None or energy_result.max_force_eV_per_A > calc_spec.max_force_eV_per_A:
            raise RuntimeError(f"Energy not found in store for train_refs[{i}] composition_map={composition_map}, occ_key={occ_key[:12]}...")
        y_true_per_prim.append(energy_result.total_energy_eV / float(n_prims))
        by_composition_map[composition_map_sig(composition_map)].append(energy_result.total_energy_eV / float(n_prims))

    # ---- Rehydrate CE and build features ----
    ce = ClusterExpansion.from_dict(cast(Mapping[str, Any], doc["payload"]))
    subspace = ce.cluster_subspace

    _, X = featurize_structures(
        subspace=subspace,
        structures=structures,
        supercell_diag=supercell_diag,
    )

    # Per-site scaling (replaceable sites only)
    prototype_spec = PrototypeSpec.from_dict(doc["prototype_spec"])
    sites_per_supercell = _n_replace_sites_from_prototype(
        prototype_spec=prototype_spec, supercell_diag=supercell_diag,
    )
    if sites_per_supercell % n_prims != 0:
        raise RuntimeError("sites_per_supercell is not divisible by n_prims (prototype/sublattice mismatch).")
    sites_per_prim = sites_per_supercell // n_prims
    scale_site = 1.0 / float(sites_per_prim)

    # ---- In-sample prediction using stored ECIs ----
    coefs = np.asarray(getattr(ce, "coefs"), dtype=np.float64)
    y_pred_per_prim_ins = predict_from_features(X, coefs)

    in_sample_recomputed = compute_stats(
        (np.array(y_true_per_prim) * scale_site).tolist(),
        (y_pred_per_prim_ins * scale_site).tolist(),
    )

    mix_rows: list[tuple[str, int, float, float, float, float]] = []
    for key, vals in sorted(by_composition_map.items(), key=lambda kv: min(kv[1])):
        n_k = len(vals)
        e_min = min(vals)
        e_max = max(vals)
        dE_prim = e_max - e_min
        # Convert to meV per site for quick comparison with CE error bars
        dE_meV_per_site = 1e3 * (dE_prim / float(sites_per_prim))
        mix_rows.append((key, n_k, e_min, e_max, dE_prim, dE_meV_per_site))

    # ---- Compare + summarize ----
    ok = (
        int(in_sample_recomputed["n"]) == int(in_sample_stored["n"])
        and math.isclose(float(in_sample_recomputed["mae_per_site"]),  float(in_sample_stored["mae_per_site"]),  rel_tol=rtol, abs_tol=atol)
        and math.isclose(float(in_sample_recomputed["rmse_per_site"]), float(in_sample_stored["rmse_per_site"]), rel_tol=rtol, abs_tol=atol)
        and math.isclose(float(in_sample_recomputed["max_abs_per_site"]), float(in_sample_stored["max_abs_per_site"]), rel_tol=rtol, abs_tol=atol)
    )

    if verbose:
        _print_summary(
            ce_key=ce_key,
            n=int(in_sample_recomputed["n"]),
            stored_in=in_sample_stored,
            recomputed_in=in_sample_recomputed,
            stored_cv=stored_cv,
            design_metrics=design_metrics,
            mixture_rows=mix_rows,
            atol=atol,
            rtol=rtol,
            ok=ok,
        )

    return ok




# verify_ce_in_sample("d0e322d8342821f7f3d3118a806e3d3df6bfd4601dab81c36d34962693bed316") # Ba2ScTaO6 r2SCAN200 0.05 DONE
# verify_ce_in_sample("2a6882a50788812be450570c7587acb73217cd0cdb77d9a66f3e77f06b0aa578") # Ca2GaNbO6 r2SCAN200 0.1 DONE
# verify_ce_in_sample("eb8e1ed0e3509423a033fe6cf54263d0fd6e0e22f31ee0b17740d57ad2ec3d7f") # Ca2GaTaO6 r2SCAN200 0.1 DONE
# verify_ce_in_sample("05c5219bc73fd62e46eb8c76582cd3c2a4f4d577eea17f24f68e3238e3af7d52") # Ca2SbScO6 r2SCAN200 0.1 DONE
# verify_ce_in_sample("d0def1e30d2b3fcfff1b41f9e96cd0671c4398fd856919ad7cfc7d6ea547eee1") # Ca2ScTaO6 r2SCAN200 0.1/0.12
# verify_ce_in_sample("f376b6bbe33c6bc0e5cfe1cda414dbd138004cbf403142f4d2f659371573e67e") # Sr2GaTaO6 r2SCAN200 0.05 FAILED
# verify_ce_in_sample("6d4a63f806171365aab94b969d3d0838b719ac57b6e70d0f91dbe34ff9a3a791") # Sr2NbScO6 r2SCAN200 0.1 DONE
# verify_ce_in_sample("046e3e63bc37dc15eb97eeae909526c0150aefda35d2626abedbc57173507cae") # Sr2ScTaO6 r2SCAN200 0.1 DONE


# verify_ce_in_sample("25ae33c4abb83c7fffa4dc76b972b11b2fec3756b9bf72ab56edd4e70dc4013a") # MgAl2O4 r2SCAN500 0.02
verify_ce_in_sample("2bcc0775cddeaac54a8422cb04f3030ee33e1b15fd2b8e2ff3b87f74ae59d264") # MgGa2O4 r2SCAN500 0.02
# verify_ce_in_sample("83e3a96ca6f2e0683e1d343a57adb9c9d250b6e0fd92be83b733602655b18a48") # ZnAl2O4 r2SCAN500 0.02

